In [2]:
import pandas as pd
import numpy as np
import flow

df = flow.get_gas_production_data()
df.head()

,State,2010_NG,2015_NG
0,AK,2826952,2804644
1,AR,927479,1010382
2,CA,273136,216772
3,CO,1589664,1688733
4,KS,325591,284184
